In [ ]:
!pip install nltk
!pip install dill
!pip install flair

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 201 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 270 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 964 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 748.8 MB 3.8 kB/s  eta 0:00:01                      | 51.5 MB 16.8 MB/s eta 0:00:42     |████████                        | 188.7 MB 22.4 MB/s eta 0:00:25/s eta 0:00:14/s eta 0:00:14     |████████████████▎               | 380.5 MB 13.5 MB/s eta 0:00:28��█████████████▋              | 410.8 MB 59.3 MB/s eta 0:00:06��██████████████              | 419.0 MB 59.3 MB/s eta 0:00:06██████████████▏           | 473.0 MB 18.5 MB/s eta 0:00:1507��███     | 633.3 MB 13.1 MB/s eta 0:00:09��█████████████▎    | 638.9 MB 13.1 MB/s eta 0:00:09��█████████████▍    | 640.6

In [ ]:
from globalfn.annotations import all_annotations
from nltk import Tree, ProbabilisticTree
import subprocess
import dill

### 1. Parsing with LX-Parser to tokenize sentences and for constituency-parsing

In [ ]:
# Parse entire text (#sentences mis-matched)
!cat input.txt | /home/jovyan/work/Tokenizer/run-Tokenizer.sh | sed '/^[[:space:]]*$/d' > output.txt
!java -Xmx500m -cp /home/jovyan/work/stanford-parser-2010-11-30/stanford-parser.jar edu.stanford.nlp.parser.lexparser.LexicalizedParser -tokenized -sentences newline -outputFormat oneline -uwModel edu.stanford.nlp.parser.lexparser.BaseUnknownWordModel cintil.ser.gz output.txt > final.txt

### 2. Convert into Constituency Trees For Further Analysis

In [ ]:
class Node:
    """
    Represent each constituent as a node
    """
    def __init__(self, value):
        """
        value: constituent values such as NP, VP, etc., or word tokens if the node is a leaf node
        children: child nodes
        idx: the words that the constituent consists of
        parent: parent node
        """
        self.value = value 
        self.children = list()
        self.idx = list()
        self.parent = None
    
    def __repr__(self):
        """
        print the node as "(self.value (...))"
        """
        if not self.children:
            return "( " + self.value + " )"
        
        s = ""
        for child in self.children:
            s += child.__repr__()
        return "( " + self.value + " " + s + " )"

class Tracker:
    """
    Used for tracking the word position as we convert the parsed string into the constituency tree.
    """
    def __init__(self, index=0):
        self.index = index

def convert_nltk_tree_into_node(T, tracker):
    """
    @Function: create the constituency tree.
    :tree: nltk tree
    :tracker: Tracker class for tracking the word positions
    :return: the root node of the constituency tree.
    """
    if type(T) is str:
        node = Node(T)
        node.idx = [tracker.index]
        tracker.index += 1
        return node

    node = Node(T.label())
    for child_tree in list(T):
        child_node = convert_nltk_tree_into_node(child_tree, tracker)
        child_node.parent = node
        node.idx += child_node.idx
        node.children += [child_node]
    return node

In [ ]:
"""
Write each tokenized sentence to output.txt for constituency-parsing. 
The sentences from input.txt are tokenized by LX-Tokenizer.
"""
sent_IDs = list()
with open("output.txt", "w+") as output_wf:
    for ID, anno in all_annotations('pt').items():
        sent_IDs.append(ID)
        with open("input.txt", 'w+') as wf:
            wf.write(anno[0].text + '\n\n')
        
        # Run LX-Tokenizer
        subprocess.run("cat /home/jovyan/work/input.txt | /home/jovyan/work/Tokenizer/run-Tokenizer.sh | sed '/^[[:space:]]*$/d' > tmp.txt", shell=True)
        
        with open("tmp.txt", 'r') as rf:
            lines = list()
            for line in rf:
                lines.append(line.strip())
            line = ' '.join(lines)
            line = line.replace('*/', "").replace("\*", "")
            output_wf.write(line + '\n')
        print(line)

Tem sido ótimo , não tem ?
Fiquei maravilhado com a coisa toda .
Em_ a verdade , estou indo embora .
Existem três tópicos abordados a_ o longo de_ a conferência que são relevantes para o que eu vou falar .
O primeiro é a extraordinária evidência de_ a criatividade humana em todas as apresentações que tivemos e em todas as pessoas presentes .
Sua variedade e multiplicidade .
O segundo é o fato de que não fazemos a menor idéia de_ o que vai acontecer em_ o futuro .
Nenhuma idéia de_ o que nos espera .
Eu me interesso por educação .
Em_ a verdade , eu descobri que todo mundo se interessa por educação .
Vocês não ?
Eu acho isso muito interessante .
Se você estiver em_ uma festa e disser que trabalha com educação ...
Em_ a verdade , você não vai a muitas festas se trabalha com educação .
Não é convidado .
E curiosamente ninguém te convida de novo .
Acho isso estranho .
Mas se você estiver , e contar para alguém ...
Quando perguntam ' O que você faz ? ' , e você responde que trabalha com edu

In [ ]:
"""
Parse the tokenized sentences with LX-Parser to obtain constituency trees.
"""
!java -Xmx500m -cp /home/jovyan/work/stanford-parser-2010-11-30/stanford-parser.jar edu.stanford.nlp.parser.lexparser.LexicalizedParser -tokenized -sentences newline -outputFormat oneline -uwModel edu.stanford.nlp.parser.lexparser.BaseUnknownWordModel /home/jovyan/work/cintil.ser.gz /home/jovyan/work/output.txt > /home/jovyan/work/final.txt

Loading parser from serialized file /home/jovyan/work/cintil.ser.gz ... done [1.1 sec].
/home/jovyan/work/output.txt
Parsing file: /home/jovyan/work/output.txt with 266 sentences.
Parsing [sent. 1 len. 7]: [Tem, sido, ótimo, ,, não, tem, ?]
Parsing [sent. 2 len. 7]: [Fiquei, maravilhado, com, a, coisa, toda, .]
Parsing [sent. 3 len. 8]: [Em_, a, verdade, ,, estou, indo, embora, .]
Parsing [sent. 4 len. 20]: [Existem, três, tópicos, abordados, a_, o, longo, de_, a, conferência, que, são, relevantes, para, o, que, eu, vou, falar, .]
Parsing [sent. 5 len. 23]: [O, primeiro, é, a, extraordinária, evidência, de_, a, criatividade, humana, em, todas, as, apresentações, que, tivemos, e, em, todas, as, pessoas, presentes, .]
Parsing [sent. 6 len. 5]: [Sua, variedade, e, multiplicidade, .]
Parsing [sent. 7 len. 21]: [O, segundo, é, o, fato, de, que, não, fazemos, a, menor, idéia, de_, o, que, vai, acontecer, em_, o, futuro, .]
Parsing [sent. 8 len. 8]: [Nenhuma, idéia, de_, o, que, nos, espera, 

In [ ]:
"""
1. Convert the output from LX-Parser into the custom Tree class for further analysis of the 
constituent-to-word and constituent-to-constituent alignments.
2. Save the result (which is the mapping of each sentence ID to the root node of the constituency tree) 
in pt_constituency_tree_dump.dill .
"""
ID_to_constituent_root_node = {}
with open("final.txt", 'r') as rf:
    for i, line in enumerate(rf):
        line = line[6:-2] # remove (ROOT prefix and ) suffix
        print(line)
        t = Tree.fromstring(line)
        tracker = Tracker()
        root = convert_nltk_tree_into_node(t, tracker)
        ID_to_constituent_root_node[sent_IDs[i]] = root

dill.dump(ID_to_constituent_root_node, open("pt_constituency_tree_dump.dill", 'wb'))

(S (S (VP (VP (V Tem) (VP (V sido) (A ótimo))) (S (PNT ,) (S (VP (ADV não) (V tem)))))) (PNT ?))
(S (S (VP (V Fiquei) (VP (V maravilhado) (PP (P com) (NP (ART a) (N coisa))))) (NP (QNT toda))) (PNT .))
(S (PP (PP (P Em_) (NP (ART a) (N verdade))) (PNT ,)) (S (VP (V estou) (NP (N' (A indo) (N' (CONJ embora) (N .)))))))
(S (S (VP (V Existem) (NP (CARD três) (N' (N' (N tópicos) (A abordados)) (PP (P a_) (NP (ART o) (N' (N' (A longo)) (PP (P de_) (NP (ART a) (N' (N conferência) (CP (NP (REL que)) (S (VP (V são) (NP (N' (N relevantes) (PP (P para) (NP (ART o) (N' (CP (NP (REL que)) (S (NP (PRS eu)) (VP (V vou) (V falar)))))))))))))))))))))) (PNT .))
(S (S (NP (ART O) (N' (ORD primeiro))) (VP (V é) (NP (ART a) (N' (N' (N extraordinária) (A evidência)) (PP (PP (P de_) (NP (ART a) (N' (N' (N criatividade) (A humana)) (PP (P em) (NP (QNT todas) (NP (ART as) (N' (N apresentações) (CP (NP (REL que)) (S (V tivemos)))))))))) (PP (CONJ e) (PP (P em) (NP (QNT todas) (NP (ART as) (N' (N pessoas) (A pr

In [3]:
# s = "(S (S (VP (VP (V Tem) (VP (V sido) (A ótimo))) (S (PNT ,) (S (VP (ADV não) (V tem)))))) (PNT ?))"
# t = Tree.fromstring(s)

In [2]:
# root.children[0].idx

In [1]:
# print(s)
# print("Display tree properties:")
# print(t.label())  # tree's constituent type
# print(t[0])  # tree's first child
# print(t[1])  # tree's second child
# print(t.height())
# print("Leaves:", t.leaves())
# print(list(t[1]))
# print(t[1, 0].label())
# print(t[1, 0, 0].label())